In [30]:
# adds a loci directory to the data folder (not here, as it will probably be useful later)
!mkdir -p ../00_data/loci

In [31]:
import ipyrad.analysis as ipa
import ipyparallel as ipp
import toytree
import toyplot
import rpy2.robjects as robjects
import os

print ('ipyrad', ipa.__version__)
print ('toyplot', toyplot.__version__)
print ('toytree', toytree.__version__)
! mb -v

# installs rpy2, which uses the power of the Dark Side to let us run R alongside Python in a Jupyter notebook.
!pip install rpy2
%load_ext rpy2.ipython

ipyrad 0.9.105
toyplot 2.0.0
toytree 3.0.10
MrBayes, Bayesian Analysis of Phylogeny

Version:   3.2.7
Features:  SSE AVX FMA Beagle readline
Host type: x86_64-conda-linux-gnu (CPU: x86_64)
Compiler:  gnu 10.3.0
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [61]:
%%R
#import vcfR to change the filtered VCF to a LOCI object
install.packages("vcfR")
library(vcfR)
install.packages("Rsamtools")
library(Rsamtools)
install.packages("pegas")
library(pegas)

* installing *source* package ‘vcfR’ ...
** package ‘vcfR’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C++ compiler: ‘x86_64-conda-linux-gnu-c++ (conda-forge gcc 11.2.0-16) 11.2.0’


x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/home/923643692/miniconda3/lib/R/include" -DNDEBUG  -I'/home/923643692/miniconda3/lib/R/library/Rcpp/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/923643692/miniconda3/include -I/home/923643692/miniconda3/include -Wl,-rpath-link,/home/923643692/miniconda3/lib    -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/923643692/miniconda3/include -fdebug-prefix-map=/workspace/croot/r-base_1695428141831/work=/usr/local/src/conda/r-base-4.3.1 -fdebug-prefix-map=/home/923643692/miniconda3=/usr/local/src/conda-prefix  -c NM2winNM.cpp -o NM2winNM.o
x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/home/923643692/miniconda3/lib/R/include" -DNDEBUG  -I'/home/923643692/miniconda3/lib/R/library/Rcpp/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/923643692/miniconda3/include -I/home/923643692/miniconda3/include 

installing to /home/923643692/miniconda3/lib/R/library/00LOCK-vcfR/00new/vcfR/libs
** R
** data
** inst
** byte-compile and prepare package for lazy loading


Creating a generic function for ‘rbind’ from package ‘base’ in package ‘vcfR’
Creating a generic function for ‘nrow’ from package ‘base’ in package ‘vcfR’


** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (vcfR)


trying URL 'https://mirror.chpc.utah.edu/pub/cran/src/contrib/vcfR_1.15.0.tar.gz'
Content type 'application/x-gzip' length 1457748 bytes (1.4 MB)
downloaded 1.4 MB


The downloaded source packages are in
	‘/tmp/RtmpXM4Bjl/downloaded_packages’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Error in library(Rsamtools) : there is no package called ‘Rsamtools’
In addition: Warning message:
package ‘Rsamtools’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages
Error in library(Rsamtools) : there is no package called ‘Rsamtools’


RInterpreterError: Failed to parse and evaluate line '#import vcfR to change the filtered VCF to a LOCI object\ninstall.packages("vcfR")\nlibrary(vcfR)\ninstall.packages("Rsamtools")\nlibrary(Rsamtools)\ninstall.packages("pegas")\nlibrary(pegas)\n'.
R error message: 'Error in library(Rsamtools) : there is no package called ‘Rsamtools’'
R stdout:
trying URL 'https://mirror.chpc.utah.edu/pub/cran/src/contrib/vcfR_1.15.0.tar.gz'
Content type 'application/x-gzip' length 1457748 bytes (1.4 MB)
==================================================
downloaded 1.4 MB


The downloaded source packages are in
	‘/tmp/RtmpXM4Bjl/downloaded_packages’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Error in library(Rsamtools) : there is no package called ‘Rsamtools’
In addition: Warning message:
package ‘Rsamtools’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org

In [9]:
# the path to your VCF formatted files
#paths
FULL_PATH   = "../00_data/00c_data_all/"
SUBSET_PATH = "../00_data/00d_data_subsets/"
LOCI_PATH   = "../00_data/loci/"
miss60_full       = FULL_PATH   + "chenopodium_20250304b-tetraploid.indivmiss60.locimiss50.thin10.maf0.01.LD0.8-1000.vcf.gz"
miss60_OAM        = SUBSET_PATH + "chenopodium_20250304b-tetraploid.indivmiss60.locimiss50.thin10.maf0.01.LD0.8-1000.no-CH-AR.vcf.gz"
miss60_OAM_OAout  = SUBSET_PATH + "chenopodium_20250304b-tetraploid.indivmiss60.locimiss50.thin10.maf0.01.LD0.8-1000.no-AR.no-CH-except-OA.vcf.gz"
vcf_files         = [miss60_full, miss60_OAM, miss60_OAM_OAout]

# passes the above variables to R
robjects.r.assign("FULL_PATH_r"  , FULL_PATH)
robjects.r.assign("SUBSET_PATH_r", SUBSET_PATH)
robjects.r.assign("LOCI_PATH_r" , LOCI_PATH)
robjects.r.assign("miss60_full_r", miss60_full)
robjects.r.assign("miss60_OAM_r" , miss60_OAM)
robjects.r.assign("vcf_files_r"  , vcf_files)

[no name],[16]
[no name],[16]
[no name],[16]


In [36]:
%%bash
# makes the temp dir, and makes unzipped copies of the .vcf.gz files in it
TMPDIR=../00_data/temp_files
mkdir -p $TMPDIR
for file in ../00_data/00c_data_all/*.gz ../00_data/00d_data_subsets/*.gz; do
    zcat $file > $TMPDIR/$(basename $file).temp
done

In [64]:
%%R
# converts the temporary .vcf files to .loci and saves them to the data/loci folder
TMPDIR<-"../00_data/temp_files/"
for (file in list.files(TMPDIR)) {
    fullpath<-paste0(TMPDIR, file)
    vcf <- read.vcfR(fullpath)  # imports the VCF
    loci <- vcfR2loci(vcf) # converts to a .loci file
    filename <- sub(".vcf.gz.temp", ".loci", basename(file)) # replaces the extension
    print(filename)
    print(paste0(LOCI_PATH_r,filename))
    write.loci(loci, file=paste0(LOCI_PATH_r,filename))
#    save(loci, file = paste0(LOCI_PATH_r,filename))
}

Scanning file to determine attributes.
File attributes:
  meta lines: 8153
  header_line: 8154
  variant count: 18736
  column count: 104
Meta line 8153 read in.
All meta lines processed.
gt matrix initialized.
Character matrix gt created.
  Character matrix gt rows: 18736
  Character matrix gt cols: 104
  skip: 0
  nrows: 18736
  row_num: 0
Processed variant: 18736
All variants processed
[1] "chenopodium_20250304b-tetraploid.indivmiss20.locimiss50.thin10.maf0.01.LD0.8-1000.loci"
[1] "../00_data/loci/chenopodium_20250304b-tetraploid.indivmiss20.locimiss50.thin10.maf0.01.LD0.8-1000.loci"
Scanning file to determine attributes.
File attributes:
  meta lines: 8192
  header_line: 8193
  variant count: 19088
  column count: 110
Meta line 8192 read in.
All meta lines processed.
gt matrix initialized.
Character matrix gt created.
  Character matrix gt rows: 19088
  Character matrix gt cols: 110
  skip: 0
  nrows: 19088
  row_num: 0
Processed variant: 19088
All variants processed
[1] "chenopodi

In [17]:
%%bash
# cleans up our temporary files
rm ../00_data/temp_files/*